# Convolutions with Complex Images (Local)

> **THIS NOTEBOOK IS STILL INCOMPLETE**

This aim is to train and optimize a CNN model to classify images of horses vs humans using TensorFlow locally.

## Overview

This tutorial demonstrates how to:

1. Train a CNN model locally using TensorFlow
2. Use the Horses or Humans dataset from TensorFlow Datasets
3. Evaluate the model on test data
4. Make predictions on individual images

### Dataset

We'll train a neural network to classify images from the [Horses or Humans](https://www.tensorflow.org/datasets/catalog/horses_or_humans) dataset.

This dataset contains 1283 images of resolution 300x300. We'll use 1027 images to train the network and 256 images to evaluate accuracy.

## Setup and Installation

In [ ]:
# Install required packages (run this if packages are not installed)
# !pip install tensorflow tensorflow-datasets matplotlib numpy

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime

# Disable progress bar for cleaner output
tfds.disable_progress_bar()

print('TensorFlow Version:', tf.__version__)
print('GPU Available:', tf.config.list_physical_devices('GPU'))

## Data Loading and Preprocessing

In [ ]:
# Define batch size
BATCH_SIZE = 32
EPOCHS = 8

# Load the dataset
print("Loading Horses or Humans dataset...")
datasets, info = tfds.load('horses_or_humans', with_info=True, as_supervised=True)

print("Dataset info:")
print(info)

# Get train and test datasets
train_dataset = datasets['train']
test_dataset = datasets['test']

print(f"Training samples: {info.splits['train'].num_examples}")
print(f"Test samples: {info.splits['test'].num_examples}")

In [ ]:
# Normalize and batch process the training dataset
train_dataset = train_dataset.map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y))
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

# Prepare test dataset for evaluation
test_dataset = test_dataset.map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y))
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

print("Data preprocessing completed.")

## Visualize Sample Data

In [ ]:
# Visualize some sample images
plt.figure(figsize=(12, 8))
class_names = ['Horse', 'Human']

# Get a batch of images and labels
for images, labels in train_dataset.take(1):
    for i in range(min(8, len(images))):
        plt.subplot(2, 4, i + 1)
        plt.imshow(images[i])
        plt.title(f'{class_names[labels[i]]}')
        plt.axis('off')

plt.tight_layout()
plt.show()

## Build the CNN Model

In [ ]:
# Build the Convolutional Neural Network
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 'horses' and 1 for 'humans'
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

# Display model architecture
model.summary()

## Train the Model

In [ ]:
# Create a callback to save the best model
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    'best_model.h5',
    monitor='val_binary_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Train the model
print("Starting training...")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset,
    callbacks=[checkpoint_callback],
    verbose=1
)

print("Training completed!")

## Visualize Training History

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot accuracy
ax1.plot(history.history['binary_accuracy'], label='Training Accuracy')
ax1.plot(history.history['val_binary_accuracy'], label='Validation Accuracy')
ax1.set_title('Model Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()

# Plot loss
ax2.plot(history.history['loss'], label='Training Loss')
ax2.plot(history.history['val_loss'], label='Validation Loss')
ax2.set_title('Model Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()

plt.tight_layout()
plt.show()

## Evaluate the Model

In [ ]:
# Evaluate the model on test data
print("Evaluating model on test data...")
test_loss, test_accuracy = model.evaluate(test_dataset, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

## Make Predictions and Visualize Results

In [ ]:
# Load test data for visualization
test_images = []
test_labels = []

# Collect some test samples
for images, labels in test_dataset.take(1):
    test_images.extend(images.numpy())
    test_labels.extend(labels.numpy())

# Convert to numpy arrays and take first 20 samples
test_images = np.array(test_images[:20])
test_labels = np.array(test_labels[:20])

# Make predictions
predictions = model.predict(test_images)
predicted_classes = (predictions > 0.5).astype(int).flatten()

print(f"Sample predictions (raw): {predictions[:5].flatten()}")
print(f"Sample predicted classes: {predicted_classes[:5]}")
print(f"Sample true labels: {test_labels[:5]}")

In [ ]:
# Visualize predictions
plt.figure(figsize=(15, 12))
class_names = ['Horse', 'Human']

for i in range(min(20, len(test_images))):
    plt.subplot(4, 5, i + 1)
    plt.imshow(test_images[i])
    
    predicted_class = predicted_classes[i]
    true_class = test_labels[i]
    confidence = predictions[i][0]
    
    # Color: green if correct, red if incorrect
    color = 'green' if predicted_class == true_class else 'red'
    
    plt.title(f'Pred: {class_names[predicted_class]}\nTrue: {class_names[true_class]}\nConf: {confidence:.3f}', 
              color=color, fontsize=10)
    plt.axis('off')

plt.tight_layout()
plt.show()

# Calculate accuracy
accuracy = np.mean(predicted_classes == test_labels)
print(f"\nAccuracy on sample: {accuracy:.3f} ({np.sum(predicted_classes == test_labels)}/{len(test_labels)})")

## Save the Model

In [ ]:
from datetime import datetime
import tensorflow as tf

# Generate timestamp for unique filenames
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save in native Keras format (recommended)
keras_path = f'horses_vs_humans_model_{timestamp}.keras'
model.save(keras_path)
print(f"Model saved to: {keras_path}")

# Save as TensorFlow SavedModel format (optional, but useful for deployment)
saved_model_path = f'horses_vs_humans_model_{timestamp}_savedmodel'
tf.saved_model.save(model, saved_model_path)
print(f"Model also saved in SavedModel format to: {saved_model_path}")


## Summary

This notebook successfully:

1. ✅ Loaded the Horses or Humans dataset from TensorFlow Datasets
2. ✅ Built and trained a CNN model locally
3. ✅ Evaluated the model on test data
4. ✅ Visualized training progress and predictions
5. ✅ Saved the trained model for future use
6. ✅ Provided a function to make predictions on new images

The model achieves good accuracy in distinguishing between horses and humans in images. You can now use this model to classify new images or further improve it with techniques like data augmentation, transfer learning, or hyperparameter tuning.